In [1]:
import torch
import torchvision
import torchdata
import random
import warnings
import sys

if not sys.warnoptions:
    warnings.simplefilter("ignore")
# Set the random seed
seed=0 # we need to set this for torch datapipe separately
random.seed(seed)

In [2]:
model_path = "/slickformer/data/models/2023_02_18_00_38_07_4cls_rn152_pr512_px1024_1440min_maskrcnn_scripting_cpu_model.pt"
scripted_model = torch.jit.load(model_path)

In [3]:
test_scenes = ["../data/tile_with_slick_512_512_3band.png"] # from cerulean cloud test fixture

In [4]:
from ceruleanml.data_pipeline import put_image_in_dict, channel_first_norm_to_tensor
source_pipe = torchdata.datapipes.iter.IterableWrapper(iterable=test_scenes)

source = (source_pipe 
    .read_tiff()
    .map(put_image_in_dict)
    .map(channel_first_norm_to_tensor)
)

In [5]:
next(iter(source))['image'].shape

torch.Size([3, 512, 512])

In [6]:
# time is an issue where there are lots of slicks
from ceruleanml.inference import mrcnn_3_class_inference
from ceruleanml.data_creation import class_dict

bbox_conf_threshold = .2
mask_conf_threshold=.2
input_size=2000
for idict in source:
    pred_dict_thresholded, pred_dict = mrcnn_3_class_inference([idict['image']], scripted_model, bbox_conf_threshold, mask_conf_threshold, input_size=input_size)
    pred_dict_thresholded_nms , pred_dict = mrcnn_3_class_inference([idict['image']], scripted_model, bbox_conf_threshold, mask_conf_threshold, input_size=input_size, interclass_nms_threshold=.5)

In [ ]:
from torchmetrics import detection

m = detection.mean_ap.MeanAveragePrecision(box_format='xyxy', iou_type='bbox', iou_thresholds=[.5], rec_thresholds=None, max_detection_thresholds=None, class_metrics=True)

m.update(preds=[pred_dicts_thresholded_nms], target=[pred_dicts_thresholded_nms])

from pprint import pprint
pprint(m.compute())

{'map': tensor(1.),
 'map_50': tensor(1.),
 'map_75': tensor(-1),
 'map_large': tensor(1.),
 'map_medium': tensor(-1.),
 'map_per_class': tensor(1.),
 'map_small': tensor(-1.),
 'mar_1': tensor(1.),
 'mar_10': tensor(1.),
 'mar_100': tensor(1.),
 'mar_100_per_class': tensor(1.),
 'mar_large': tensor(1.),
 'mar_medium': tensor(-1.),
 'mar_small': tensor(-1.)}


In [62]:
import httpx
import configparser
import base64
# parses secrets so we don't expose them in the notebook
config = configparser.ConfigParser()
config.read('../.secrets')
config = config['secrets']
# reads image as bytes, converts bytes to string so it can be sent as a post request
with open(test_scenes[0], 'rb') as f:
    byte_string = f.read()
    base64_string = base64.b64encode(byte_string).decode('utf-8')
# we format the payload and headers according to what the docs expect
# docs are at f"{config['CERULEAN_STAGING_URL']}/docs"
predict_url = f"{config['CERULEAN_STAGING_URL']}/predict"
payload = {"stack": [{"image": base64_string, "bounds":[ 0,0,0,0 ]}]}
headers = {"Authorization": f"Bearer {config['CERULEAN_API_KEY']}"}
response = httpx.post(predict_url, json=payload, headers=headers, timeout=None)

In [71]:
response.json()['stack'][0]['features'][0]['properties']

{'classification': 3, 'confidence': 0.8743406534194946}

In [73]:
print(pred_dict_thresholded_nms['labels'], pred_dict_thresholded_nms['scores'])

tensor([3]) tensor([0.8743])
